In [33]:
import csv
import re
import os, sys
import shelve
from isort import stream
from majed_modules.multi import mysort
from poppler import load_from_file
from pathlib import Path
from openpyxl import Workbook, load_workbook
from openpyxl.styles import stylesheet
from openpyxl.formatting import formatting
from majed_modules.multi import basic_salary_dictionary

In [34]:
def mysort(item):
    date_regex = re.compile(r"Payroll for (\d*)/ (\d{4})")
    date = date_regex.search(item)
    m = date.group(1)
    y = date.group(2)
    m_y = y[2:4] + m
    return int(m_y)

In [35]:
def main(stream):
    '''
    Insert a path and returns sorted pages list
    '''
    list_of_all_pages = []
    for p in Path(stream).iterdir():
        if p.suffix == '.pdf':
            doc = load_from_file(p)
        if doc.pages == 2:
            dual0 = doc.create_page(0)
            dual1 = doc.create_page(1)
            txt1 = dual0.text()
            txt2 = dual1.text()
            mix = txt1 + txt2
            list_of_all_pages.append(mix)
        else:
            singel = doc.create_page(0)
            txt = singel.text()
            list_of_all_pages.append(txt)

    list_of_all_pages.sort(key=mysort)
    return list_of_all_pages


In [36]:
path = ''
allpages = main('./Assets/Waleed Moasbil')

In [37]:
def basic_salary_dictionary(args):
    Basic_Salary_Dictionary = {}
    for bs in args:
        # bsx = re.search(r'^0001[\s]*(.*?)[\s]*([\d,.]*)[\s]*(\d{6})', bs,
        # re.MULTILINE)
        for l in bs.splitlines():
            if re.search(r'^0001', l):
                s = l.split()
                bs_amount = s[-2]
                bs_amount = float(bs_amount.replace(',', ''))
                bs_date = s[-1]
                if bs_date not in Basic_Salary_Dictionary:
                    Basic_Salary_Dictionary[bs_date] = round(bs_amount, 3)

    return Basic_Salary_Dictionary


bsd = basic_salary_dictionary(allpages)


In [38]:
total = 0
for page in allpages:
    payroll = re.search(r'Payroll for (\d*/ \d{4})', page)
    annual = re.search(r'^3000.+', page, re.MULTILINE)
    modefied_payroll = payroll.group(1).replace(('/'), '')
    modefied_payroll = modefied_payroll.replace((' '), '')
    m = modefied_payroll[0:2]
    y = modefied_payroll[2:]
    m_y = y + m
    count = 0
    aux = []
   
    for line in page.splitlines():
        s = line.split()
        if re.search(r'^1110|^1111|^1115|^1113|^1320|^1315|^3000', line):
            if re.search(r'^1315.+-', line):
                wage = s[1]
                disc = "nature of work 5%"
                amount = float(s[-2].replace(',', ''))
                date = s[-1]
                if m_y not in date:
                    if "201903" not in line:
                        total+=amount
                        print(
                        f'{wage} {disc}                                                      {amount}     {date}  out'
                    )
            if annual:
                sannual = annual.group().split()
                if re.search(r'^1315', line):
                    wage = s[1]
                    disc = u"بدل طبيعة عمل ٥٪"
                    amount = float(s[-2].replace(',', ''))
                    date = s[-1]
                    if m_y in date:
                        if "201903" not in line:
                            bsd315 = bsd.get(date)* 0.15
                            dif = round(amount - bsd315, 2)
                            total+=dif
                            print(
                            f'{wage} {disc}                           {sannual[-1]}                           {dif}     {date}  in'
                        )




total

Nature nature of work 5%                           1.00                           0.0     201407  in
Nature nature of work 5%                                                      -83.43     201406  out
Nature nature of work 5%                                                      -40.37     201407  out
Nature nature of work 5%                           5.00                           -208.58     201411  in
Nature nature of work 5%                           1.00                           -40.37     201412  in
Nature nature of work 5%                           5.00                           -223.47     201502  in
Nature nature of work 5%                                                      -40.37     201503  out
Nature nature of work 5%                           3.00                           -133.9     201506  in
Nature nature of work 5%                           2.00                           -86.39     201508  in
Nature nature of work 5%                                                  

-8929.560000000001